<a href="https://colab.research.google.com/github/himoyuzuki/colab_analytics/blob/main/OpenStreetMap%E3%81%A7%E3%82%B3%E3%83%B3%E3%83%93%E3%83%8B%E3%81%AE%E6%95%B0%E3%82%92%E8%AA%BF%E3%81%B9%E3%81%A6%E3%81%BF%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenStreetMapでコンビニの数を調べてみる

In [1]:
!pip install osmnx geopandas folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.4 MB/s eta 0:00:00


In [30]:
from functools import reduce
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium
import plotly.express as px
from scipy import stats

In [24]:
# 日本の市区町村から代表地点をランダム選定
region_list = [
    "札幌市", "函館市", "仙台市", "会津若松市", "横手市", "渋谷区",
    "横浜市", "つくば市", "日光市", "みなかみ町", "新潟市", "金沢市",
    "名古屋市", "高山市", "飯田市", "大阪市", "京都市", "神戸市",
    "鳥取市", "出雲市", "広島市", "松山市", "四万十市", "福岡市",
    "長崎市", "別府市", "阿蘇市", "霧島市", "那覇市", "石垣市"
]

# 代表地点の人口。Google検索結果からおおよその値を抽出。調査年度について若干ずれがあり、おおよその値
population_list = [
    "1,973,395", "243,942", "1,096,704", "118,159", "82,553", "243,883",
    "3,778,318", "255,639", "77,027", "18,383", "789,284", "460,000",
    "2,300,000", "84,092", "98,164", "2,752,412", "1,463,723", "1,500,000",
    "186,439", "150,000", "1,173,543", "505,948", "32,694", "1,612,392",
    "408,000", "122,000", "24,930", "123,812", "300,000", "49,000"
]

# OpenStreetMapでの検索ワード
place_list = [
    "Sapporo, Hokkaido, Japan", "Hakodate, Hokkaido, Japan", "Sendai, Miyagi, Japan",
    "Aizuwakamatsu, Fukushima, Japan", "Yokote, Akita, Japan", "Shibuya, Tokyo, Japan",
    "Yokohama, Kanagawa, Japan", "Tsukuba, Ibaraki, Japan", "Nikko, Tochigi, Japan",
    "Minakami, Gunma, Japan", "Niigata, Niigata, Japan", "Kanazawa, Ishikawa, Japan",
    "Nagoya, Aichi, Japan", "Takayama, Gifu, Japan", "Iida, Nagano, Japan",
    "Osaka, Osaka, Japan", "Kyoto, Kyoto, Japan", "Kobe, Hyogo, Japan",
    "Tottori, Tottori, Japan", "Izumo, Shimane, Japan", "Hiroshima, Hiroshima, Japan",
    "Matsuyama, Ehime, Japan", "Shimanto, Kochi, Japan", "Fukuoka, Fukuoka, Japan",
    "Nagasaki, Nagasaki, Japan", "Beppu, Oita, Japan", "Aso, Kumamoto, Japan",
    "Kirishima, Kagoshima, Japan", "Naha, Okinawa, Japan", "Ishigaki, Okinawa, Japan"
]

region_info = pd.DataFrame({
    'Region': region_list,
    'Population': population_list,
    'Search Words': place_list
})

region_info['Population'] = region_info['Population'].str.replace(',', '').astype(int)

region_info

,Region,Population,Search Words
0,札幌市,1973395,"Sapporo, Hokkaido, Japan"
1,函館市,243942,"Hakodate, Hokkaido, Japan"
2,仙台市,1096704,"Sendai, Miyagi, Japan"
3,会津若松市,118159,"Aizuwakamatsu, Fukushima, Japan"
4,横手市,82553,"Yokote, Akita, Japan"
5,渋谷区,243883,"Shibuya, Tokyo, Japan"
6,横浜市,3778318,"Yokohama, Kanagawa, Japan"
7,つくば市,255639,"Tsukuba, Ibaraki, Japan"
8,日光市,77027,"Nikko, Tochigi, Japan"
9,みなかみ町,18383,"Minakami, Gunma, Japan"


In [4]:
# OpenStreetMapから「コンビニ」のデータを取得
tags = {"shop": "convenience"}
df_list = []

for place_name in region_info['Search Words'].tolist():
    gdf = ox.features.features_from_place(place_name, tags)

    print(f"{place_name}内で{len(gdf)}軒のコンビニが見つかりました。")

    # 地図の中心を計算
    center_lat = gdf.unary_union.centroid.y
    center_lon = gdf.unary_union.centroid.x

    # foliumで地図を作成
    m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

    # 取得した場所にマーカーをプロット
    for idx, row in gdf.iterrows():
        # 'name'列が存在すればポップアップに表示、なければ'shop'を表示
        popup_text = row.get('name', 'その他')

        # geometryが点でも多角形でも、その重心を計算して座標を取得する
        centroid = row.geometry.centroid
        lat = centroid.y
        lon = centroid.x

        folium.Marker(
            location=[lat, lon], # 修正した座標を使用
            popup=popup_text,
            icon=folium.Icon(color='blue', icon='shopping-cart')
        ).add_to(m)

    # 地図をHTMLファイルとして保存 (ファイル名もエリアに合わせました)
    m.save(f"{place_name}_{tags['shop']}.html")

    # コンビニの数をカウント
    cnt_df = gdf['name'].value_counts(dropna=False).reset_index()
    cnt_df['count'] = pd.to_numeric(cnt_df['count'], errors='coerce')
    cnt_df = cnt_df.rename(columns={'count' : f"count_{place_name}"})
    df_list.append(cnt_df)

    print(f"地図が'{place_name}_{tags['shop']}.html'として保存されました。")

Sapporo, Hokkaido, Japan内で972軒のコンビニが見つかりました。


/tmp/ipython-input-682028865.py:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center_lat = gdf.unary_union.centroid.y
/tmp/ipython-input-682028865.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center_lon = gdf.unary_union.centroid.x


地図が'Sapporo, Hokkaido, Japan_convenience.html'として保存されました。
Hakodate, Hokkaido, Japan内で102軒のコンビニが見つかりました。


/tmp/ipython-input-682028865.py:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center_lat = gdf.unary_union.centroid.y
/tmp/ipython-input-682028865.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center_lon = gdf.unary_union.centroid.x


地図が'Hakodate, Hokkaido, Japan_convenience.html'として保存されました。
Sendai, Miyagi, Japan内で561軒のコンビニが見つかりました。


...

/tmp/ipython-input-682028865.py:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center_lat = gdf.unary_union.centroid.y
/tmp/ipython-input-682028865.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center_lon = gdf.unary_union.centroid.x


Naha, Okinawa, Japan内で197軒のコンビニが見つかりました。


/tmp/ipython-input-682028865.py:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center_lat = gdf.unary_union.centroid.y
/tmp/ipython-input-682028865.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center_lon = gdf.unary_union.centroid.x


地図が'Naha, Okinawa, Japan_convenience.html'として保存されました。
Ishigaki, Okinawa, Japan内で17軒のコンビニが見つかりました。
地図が'Ishigaki, Okinawa, Japan_convenience.html'として保存されました。


/tmp/ipython-input-682028865.py:11: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center_lat = gdf.unary_union.centroid.y
/tmp/ipython-input-682028865.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center_lon = gdf.unary_union.centroid.x


In [5]:
# コンビニのカウントDataFrameをマージ
merge_func = lambda left, right: pd.merge(left, right, on='name', how='outer')
conv_df = reduce(merge_func, df_list)  # reduceを使ってリスト内のDataFrameを順番にマージ
conv_df = conv_df.fillna(0)

conv_df.head()

,name,"count_Sapporo, Hokkaido, Japan","count_Hakodate, Hokkaido, Japan","count_Sendai, Miyagi, Japan","count_Aizuwakamatsu, Fukushima, Japan","count_Yokote, Akita, Japan","count_Shibuya, Tokyo, Japan","count_Yokohama, Kanagawa, Japan","count_Tsukuba, Ibaraki, Japan","count_Nikko, Tochigi, Japan",...,"count_Hiroshima, Hiroshima, Japan","count_Matsuyama, Ehime, Japan","count_Shimanto, Kochi, Japan","count_Fukuoka, Fukuoka, Japan","count_Nagasaki, Nagasaki, Japan","count_Beppu, Oita, Japan","count_Aso, Kumamoto, Japan","count_Kirishima, Kagoshima, Japan","count_Naha, Okinawa, Japan","count_Ishigaki, Okinawa, Japan"
0,100円ショップ・シルク松山椿店,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100円ローソン,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3COINS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3coins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7 Eleven,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# 一次保存
conv_df.to_csv("/content/conv_df.csv", index=False)

In [8]:
# 位置時保存ファイルの読み込み
# conv_df = pd.read_csv("/content/conv_df.csv")
# conv_df.head()

,name,"count_Sapporo, Hokkaido, Japan","count_Hakodate, Hokkaido, Japan","count_Sendai, Miyagi, Japan","count_Aizuwakamatsu, Fukushima, Japan","count_Yokote, Akita, Japan","count_Shibuya, Tokyo, Japan","count_Yokohama, Kanagawa, Japan","count_Tsukuba, Ibaraki, Japan","count_Nikko, Tochigi, Japan",...,"count_Hiroshima, Hiroshima, Japan","count_Matsuyama, Ehime, Japan","count_Shimanto, Kochi, Japan","count_Fukuoka, Fukuoka, Japan","count_Nagasaki, Nagasaki, Japan","count_Beppu, Oita, Japan","count_Aso, Kumamoto, Japan","count_Kirishima, Kagoshima, Japan","count_Naha, Okinawa, Japan","count_Ishigaki, Okinawa, Japan"
0,100円ショップ・シルク松山椿店,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100円ローソン,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3COINS,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3coins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7 Eleven,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
total_counts = [conv_df[f"count_{place}"].sum() for place in place_list]
region_info['Total Count'] = total_counts
region_info

,Region,Population,Search Words,Total Count
0,札幌市,1973395,"Sapporo, Hokkaido, Japan",972.0
1,函館市,243942,"Hakodate, Hokkaido, Japan",102.0
2,仙台市,1096704,"Sendai, Miyagi, Japan",561.0
3,会津若松市,118159,"Aizuwakamatsu, Fukushima, Japan",68.0
4,横手市,82553,"Yokote, Akita, Japan",46.0
5,渋谷区,243883,"Shibuya, Tokyo, Japan",252.0
6,横浜市,3778318,"Yokohama, Kanagawa, Japan",1316.0
7,つくば市,255639,"Tsukuba, Ibaraki, Japan",126.0
8,日光市,77027,"Nikko, Tochigi, Japan",44.0
9,みなかみ町,18383,"Minakami, Gunma, Japan",13.0


In [27]:
fig = px.scatter(
    region_info,
    x='Population',
    y='Total Count',
    title='人口と合計カウントの関係性',
    labels={'Population': '人口 (人)', 'Total Count': '合計カウント'},
    hover_data=['Region'],  # 点にカーソルを合わせると都市名が表示される
    trendline='ols'  # 近似直線（線形回帰）を追加
)
fig.show()

In [31]:
correlation_coefficient, p_value = stats.pearsonr(region_info['Population'], region_info['Total Count'])

print(f"ピアソンの相関係数: {correlation_coefficient:.4f}")
print(f"p値: {p_value}")

ピアソンの相関係数: 0.9642
p値: 1.1149573102550494e-17


**まとめ**

日本の市区町村について、ランダムに30箇所をピックアップして調査したところ、コンビニの数と人口には0.96という強い相関が見られた